In [1]:
import json
from definitions import ROOT_DIR
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import plotly.graph_objs as go


In [2]:
eval_dir = ROOT_DIR / "output" / "offline_evaluation"


In [3]:
def plot_results(experiment_name, fig=None, cmap='Viridis', name="Heuristic", opacity=0.6):
    result_path = eval_dir / f"{experiment_name}.json"
    with open(result_path, "r") as f:
        data = json.load(f)
    scores = np.array([item['score'] for item in data])
    cpa_constraints = np.array([item['cpa_constraint'] for item in data])
    budgets = np.array([item['budget'] for item in data])
    
    # Create grid and interpolate
    grid_x, grid_y = np.mgrid[min(cpa_constraints):max(cpa_constraints):100j,
                            min(budgets):max(budgets):100j]
    grid_z = griddata((cpa_constraints, budgets), scores, (grid_x, grid_y), method='cubic')

    if fig is None:
        fig = go.Figure(data=[go.Surface(z=grid_z, x=grid_x, y=grid_y, colorscale=cmap, opacity=opacity,
                        colorbar=dict(title=name, len=0.5, y=0.25))])
        fig.update_layout(scene = dict(
                        xaxis_title='CPA Constraint',
                        yaxis_title='Budget',
                        zaxis_title='Score'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))
    else:
        fig.add_trace(go.Surface(z=grid_z, x=grid_x, y=grid_y, colorscale=cmap, opacity=opacity,
                      colorbar=dict(title=name, len=0.5, y=0.75)))
    return fig

In [ ]:
# Load all the evaluations from one experiment and plot the means
# experiment_name = "IQL/train_quantile_16_002"
experiment_name = "BC/027_train_full_dataset_29"
# experiment_name = "IQL/train_full_dataset_16_001"
experiment_dir = eval_dir / experiment_name

# Iterate through all the json files inthe directory
checkpoint_paths = list(experiment_dir.glob("*.json"))

def get_number(checkpoint_path):
    return int(checkpoint_path.stem.split("_")[-1])

sorted_checkpoints = sorted(checkpoint_paths, key=get_number)
mean_score_list = []
for path in sorted_checkpoints:
    result = json.load(open(path, "r"))
    score_list = [item['score'] for item in result]
    mean_score_list.append(np.mean(score_list))

plt.plot([get_number(c) for c in sorted_checkpoints], mean_score_list)

# Get the best checkpoint
best_checkpoint = sorted_checkpoints[np.argmax(mean_score_list)]
print(best_checkpoint, np.max(mean_score_list))


In [ ]:
opacity = 1
algo = "PPO"

# checkpoint = 19000
# print(f"Checkpoint: {checkpoint}")
# experiment_name = f"BC/train_whole_dataset_29_017/checkpoint_{checkpoint}"  # "onlineLpTest"  # BC_train_004_checkpoint_190000
# experiment_name = f"BC/train_full_dataset_16_023/checkpoint_{checkpoint}"  # "onlineLpTest"  # BC_train_004_checkpoint_190000
# experiment_name = f"BC/train_top_quantile_16_024/checkpoint_4000"  # good, aslo 5000
# experiment_name = f"IQL/train_full_dataset_16_001/checkpoint_21000"  # good
# experiment_name = f"IQL/train_regression_16_003/checkpoint_36000"  # good
# experiment_name = f"BC/027_train_full_dataset_29/checkpoint_3000"  # good
experiment_name = f"PPO/033_ppo_seed_0_dense_base_ranges_29_obs_exp_multi_action/checkpoint_3500000"
# experiment_name = f"PPO/029_ppo_seed_0_dense_base_ranges_29_obs_exp_single_action_simplified/checkpoint_10750000"  # best, 0.41
# experiment_name = f"PPO/031_ppo_seed_0_dense_base_ranges_19_obs_exp_multi_action_simplified/checkpoint_8000000"  # best, 0.41
# experiment_name = f"BC/train_top_regression_16_025/checkpoint_{checkpoint}"  # good 16000, 19000 current best at 0.39
fig = plot_results(experiment_name, cmap="Blues_r", name="Competitor", opacity=opacity)

# experiment_name = "onlineLpTest/onlineLpTest/checkpoint_"  # "heuristic"  
experiment_name = f"BC/train_top_regression_16_025/checkpoint_19000" # best
# experiment_name = f"IQL/train_regression_16_002/checkpoint_9000"  # good

fig = plot_results(experiment_name, fig=fig, cmap='Reds', name="Sota", opacity=opacity)
fig.show()

In [ ]:
# experiment_name = "BC_train_004_checkpoint_27000"  # best so far?
for idx in range(1, 5):
    checkpoint = int(idx * 1000)
    print(f"Checkpoint: {checkpoint}")
    experiment_name = f"IQL_train_regression_001_checkpoint_{checkpoint}"  # "onlineLpTest"  # BC_train_004_checkpoint_190000
    fig = plot_results(experiment_name, cmap="Turbo_r", name="IQL")
    experiment_name = "onlineLpTest"  # "onlineLpTest"  # BC_train_004_checkpoint_190000
    fig = plot_results(experiment_name, fig=fig, cmap='Inferno', name="Heuristic")
    fig.show()

In [ ]:

result_path = eval_dir / f"{experiment_name}.json"
with open(result_path, "r") as f:
    data = json.load(f)

scores = np.array([item['score'] for item in data])
cpa_constraints = np.array([item['cpa_constraint'] for item in data])
budgets = np.array([item['budget'] for item in data])

# Create grid and interpolate
grid_x, grid_y = np.mgrid[min(cpa_constraints):max(cpa_constraints):100j,
                          min(budgets):max(budgets):100j]
grid_z = griddata((cpa_constraints, budgets), scores, (grid_x, grid_y), method='cubic')

# Plotly surface plot
fig = go.Figure(data=[go.Surface(z=grid_z, x=grid_x, y=grid_y, colorscale='Viridis')])

# Set the axes ranges explicitly if needed
fig.update_layout(scene = dict(
                    xaxis=dict(range=[min(cpa_constraints), max(cpa_constraints)], title='CPA Constraint'),
                    yaxis=dict(range=[min(budgets), max(budgets)], title='Budget'),
                    zaxis=dict(title='Score')),
                  width=700,
                  margin=dict(r=20, b=10, l=10, t=10))

fig.show()

In [ ]:
import plotly.graph_objs as go
import numpy as np

# # Generate some example data
# x = np.linspace(-5, 5, 100)
# y = np.linspace(-5, 5, 100)
# x, y = np.meshgrid(x, y)
# z = np.sin(np.sqrt(x**2 + y**2))

# Create the surface plot
fig = go.Figure(data=[go.Surface(z=grid_z, x=grid_x, y=grid_y, colorscale='Viridis')])

# Update layout to include axis labels
fig.update_layout(scene=dict(
                    xaxis_title='X Axis',
                    yaxis_title='Y Axis',
                    zaxis_title='Z Axis'),
                  width=700,
                  margin=dict(r=20, b=10, l=10, t=10))

# Show the plot
fig.show()


In [ ]:
experiment_name = "BC_train_004_checkpoint_100000"  # "onlineLpTest"  # BC_train_004_checkpoint_190000
plot_results(experiment_name)

In [ ]:
experiment_name = "BC_train_004_checkpoint_190000"  # "onlineLpTest"  # BC_train_004_checkpoint_190000
plot_results(experiment_name)

In [ ]:
experiment_name = "onlineLpTest"  # "onlineLpTest"  # BC_train_004_checkpoint_190000
plot_results(experiment_name)